In [1]:
import os
import json
from pyspark.sql import SparkSession

os.environ["HADOOP_HOME"] = r"C:\hadoop"
os.environ["hadoop.home.dir"] = r"C:\hadoop"
os.environ["PATH"] += os.pathsep + r"C:\hadoop\bin"

os.environ['JAVA_HOME'] = r"D:\Applns\Amazon Corretto\jdk17.0.15_6"
os.environ['PATH'] = r"D:\Applns\Amazon Corretto\jdk17.0.15_6\bin" + os.pathsep + os.environ['PATH']


In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("RumbleVideoETL_bronzelayer") \
    .config("spark.ui.port", "4050") \
    .getOrCreate()
spark

In [3]:

with open("../data/raw/rumble_video_data.json") as f:
    raw_data = json.load(f)

flat_data = []
for vid, info in raw_data.items():
    info["video_id"] = vid
    flat_data.append(info)

with open("../data/raw/flattened_rumble.json", "w") as f:
    json.dump(flat_data, f, indent=2)


In [5]:
flattened_data = spark.read.option("multiline", "true").json("../data/raw/flattened_rumble.json")
flattened_data.show(n=100)
flattened_data.write.mode("overwrite").parquet("../data/bronze/rumble_videos_bronze.parquet")



+--------------------+--------------------+---------------+--------------------+-------------+--------------------+--------------------+--------------------+----------------------+--------------+----------------+-----------------+--------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------+
|        channel_name|            comments|downvotes_count|          transcript|upvotes_count|video_comment_number|          video_date|   video_description|video_description_cont|video_duration|video_duration_s|       video_host|video_id|    video_source_url|          video_tags| video_thumbnail_url|         video_title|           video_url|video_views|
+--------------------+--------------------+---------------+--------------------+-------------+--------------------+--------------------+--------------------+----------------------+--------------+----------------+-----------------+--------+--------------------+--------------------+-----

In [6]:

table = spark.read.parquet("../l0_table.parquet")
table.show(truncate=False)


+------------+----------------+-----------------+-----------------+-------------------+------------+----------------+------------------+------------------------------+---------------------------+-------------+-----------------------+------------------------+---------------------+---------------------------+----------------------------+---------------+-----------------------------+--------------------------+--------------------------------+-------------------------+------------+--------------------+------------------------------+--------------------------------+-------------+----------------------+----------------------------------+------------------------+------------------------------------------+------------------------+-------------------+-----------------------------------+---------------------------------------------+-----------------------------------+--------------+-------------+--------------+----------------+---------------------+-----------------------------------------------

In [7]:
spark.stop()